##### <font color='darkblue'>Week 8 - Lab Notebook - Part 2 - Working with Networks</font>

- November 2023
- https://https://github.com/tisimpson/bioinformatics1
- ian.simpson@ed.ac.uk

##### <font color='darkblue'>Introduction</font>
In this computing lab we’re going to be putting together what we've learned about biological databases and ontologies to do some summary analysis of genes invovled in the "Dopaminergnic Synapse" pathway. These can all be done using the KEGG and String-DB websites directly but we will show here that there is much greater power and flexibility available when you start using programmatic methods to carefully control your analyses.

##### <font color='darkblue'>Learning Outcomes</font>
After this tutorial you should be comfortable with:
- Retrieving pathway information from KEGG
- converting between accession IDs of different databases
- Retrieving protein-protein interaction and network data from String-DB
- Automating these processes using APIs and the NetworkX, and GSEAPy python packages

##### <font color='darkblue'>Setting up the Environment & Retrieving Data</font>

In [ ]:
# Setting Up the Programming Environment
# %pip install networkx
# %pip install gseapy

# import modules for use in the notebook

# handling www based requests (like APIs)
import urllib as ul

# standard Python data handling modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

# working with networks
import networkx as nx

# working with geen set enrichment analysis (GSEA)
import gseapy as gp

For Step1 you can either use the fully automated approcah using Steps 1a, and 1b, or working from files you generate from the KEGG and String-DB websites as described in Step 1c.

##### <font color='darkblue'>Fetch KEGG Pathway Information</font>

In [ ]:
# Now we will use this file which contains the full pathway details including the gene names.

# open the file
dop_file = open('../data/pathways/dop_synapse.txt','r')

# I wanted to show you some basic python parsing and a simple for loop with a conditional in to demonstrate how you can quickly build simple parsers.
# There are quicker ways to do this, but this is a good learning example.

# create an empty dataframe with two columns
dop_df = pd.DataFrame(columns=['gene_id','gene_symbol','description'])

# set a flag for our parser
flag=0

# work through the text file one line at a time
for line in dop_file:
    # find the start of the gene entries
    if 'GENE' in line:
        # add the first gene tp the dataframe
        gene_id,remain = line.strip('GENE').strip().split('  ')
        gene_symbol,description = remain.split(';')
        # add a new row to the dataframe containing the gene_id and description
        dop_df = pd.concat([dop_df,pd.DataFrame([[gene_id,gene_symbol,description]],columns=['gene_id','gene_symbol','description'])],ignore_index=True)
        # set the flag to 1, we are in the gene section of the file
        flag = 1
    # stop when we reach the end of the section and escape the file
    elif 'COMPOUND' in line:
        break
    # continue adding the genes to the dataframe
    elif flag == 1:
        gene_id,remain = line.strip('GENE').strip().split('  ')
        gene_symbol,description = remain.split(';')
        # add the gene to the dataframe
        dop_df = pd.concat([dop_df,pd.DataFrame([[gene_id,gene_symbol,description]],columns=['gene_id','gene_symbol','description'])],ignore_index=True)

# close the file
dop_file.close()

# view the file
dop_df.head()

# you now have the gene_ids (NCBI EntrezIDs for the genes in the pathway)
print('The Dopaminergic Synapse pathway has '+str(dop_df.shape[0])+' genes in it.\n')

# store the gene_ids in a numpy array
gene_ids = dop_df['gene_id'].to_numpy()

gene_symbols = dop_df['gene_symbol'].to_numpy()

# show the gene_ids
print(gene_ids)

# write the gene_ids to a file
with open('../data/pathways/dop_geneids.txt','w') as f:
    for gene_id in gene_ids:
        f.write(gene_id+'\n')

# show the gene_symbols
print(gene_symbols)

# write the gene_symbols to a file
with open('../data/pathways/dop_symbols.txt','w') as f:
    for symbol in gene_symbols:
        f.write(symbol+'\n')

##### <font color='darkblue'>Retrieval of Protein-Protein Interactions from STRING</font>

The details of the String-DB API can be found here - [https://string-db.org/help/api/](https://string-db.org/help/api/)

APIs have specific formats required for their query URLs and it getting these correct in your code can take a little time until you get used to them. In this case we need to concatenate (stitch together) our gene IDs using a '%0D' string. This is actually the encoding for a line-return which is in effect mimicking the one gene per line entry that you would paste into the web page.

In [ ]:
# create a concatenated list of entrezIDs as strings
# note we are taking integer gene_ids from the 'gene_id' column of the dataframe we generated above then using
# the map function to convert each one into a string. The join function then concatenates them using the '%0D' string
# to stitch them all together. This string will be used to help us build the API query URL.
entrezIDs = '%0D'.join(map(str,dop_df['gene_id']))

# pass the list of EntrezIDs to the String-DB API return the String-IDs
# we first form the query url using the 'get_string_ids' API function which takes a list of identifiers and
# converts them into the internal String-DB accession IDs. This massively speeds up the search and allows us to
# search for more than 10 at once which is an API restriction for other API functions if String-DB internal accessions 
# aren't used.
query_url = 'https://string-db.org/api/tsv-no-header/get_string_ids?identifiers='+entrezIDs+'&species=9606&format=only-ids'

# use the urllib library to retrieve the String-DB internal IDs
result = ul.request.urlopen(query_url).read().decode('utf-8')

# now we want to query String-DB to retrieve interactions from this list of String-DB IDs
# we create a concatenated list of stringdbIDs in much the same way as above for the Entrez Gene IDs
stringdbIDs = '%0D'.join(result.splitlines())

# again we build the query for interactions using the String-DB IDs
query_url = 'https://string-db.org/api/tsv/network?identifiers='+stringdbIDs+'&species=9606'

# again using urllib to retrieve the interactions these are returned in a standard tab delimied text format
interactions = ul.request.urlopen(query_url).read().decode('utf-8').splitlines()

# we need to split the result by these 'tabs' (\t - is used to identfy them)
int_test = [interaction.split('\t') for interaction in interactions]

# we extract the field names from the first row
column_names = int_test[:1][0]

# create a Pandas dataframe of the interaction data we have just retrieved from String-DB
interactions_df = pd.DataFrame(int_test,columns=column_names)

# delete the first row that held the fieldnames but we no longer need
interactions_df = interactions_df.drop(labels=0,axis=0)

# remove any duplicate rows
final_interactions = interactions_df.drop_duplicates()

# show the top of the protein-protein interaction table
final_interactions.head()

##### <font color='darkblue'>Generating the Protein-Protein Interaction Network</font>

Next we are going to use the NetworkX Python library to create the protein-protein interaction network.

NetworkX - Network Analysis in Python - [https://networkx.org/documentation/stable/index.html](https://networkx.org/documentation/stable/index.html)

In [ ]:
# create a simple network view of the interactions using the NetworkX library
# https://networkx.org/documentation/stable/index.html

import networkx as nx
import matplotlib.pyplot as plt

 #Create an empty graph
G = nx.Graph()

# add all nodes
G.add_nodes_from(set(final_interactions['preferredName_A']) | set(final_interactions['preferredName_B'])) 

# add the edges (connections) to the network
edges = []
for edge1 , edge2  in zip(final_interactions['preferredName_A'] , final_interactions['preferredName_B']) : #add all edge to the network
    edges.append((edge1 , edge2 ))
G.add_edges_from(edges)

# draw the network with a force directed layout specify plot size and node size and thin light gray edges
plt.figure(figsize=(15,15))
nx.draw(G, pos=nx.spring_layout(G,k=2), with_labels=True,node_size=100,edge_color='gray',node_color='lightblue',width=0.5)

##### <font color='darkblue'>Node Degree</font>

Node degree is simply the number of connections a node has. Nodes with higher degree are often called `hub` nodes because they have many connections to other members of the network.

In [ ]:
#sort the genes (node names) by degree
sorted_list = sorted(G.degree(), key=lambda item: item[1] , reverse=True)

# print out the top10 using prettytable
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Gene","Degree"]
for gene in sorted_list[:10]:
    x.add_row(gene)
print(x)

##### <font color='darkblue'>Closeness Centrality</font>
This is a measure of how close a node is to the centre of the network. The closer a node is to the centre the shorter its path to all other nodes and hence its more likely to be representative of the network

In [ ]:
# sort the genes (node names) by proximity to center
sorted_list = sorted(nx.closeness_centrality(G).items(), key=lambda item: item[1] , reverse=True) 

# print out the top10 using prettytable
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Gene","Closeness"]
for gene in sorted_list[:10]:
    x.add_row(gene)
print(x)

##### <font color='darkblue'>Clustering Coefficient</font>
The clustering coefficient is a measure which combines centrality and degree. It measures the number of triangles a node can form ('the friend of my friend is my friend'). If a node has more common friends with other nodes it more likely to representative of the network

In [ ]:
# sort the genes (node names) by clustering coefficient
sorted_list = sorted(nx.clustering(G).items(), key=lambda item: item[1] , reverse=True)

# print out the top10 using prettytable
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Gene","Clustering"]
for gene in sorted_list[:10]:
    x.add_row(gene)
print(x)

##### <font color='darkblue'>Clustering the Network</font>

In [ ]:
import networkx as nx
from prettytable import PrettyTable

# we're going to cluster the networkx modularity clustering algorithm
communities = nx.algorithms.community.modularity_max.greedy_modularity_communities(G)

# print the number of communities
print('The network has '+str(len(communities))+' communities.\n')

# create sub-grpahs for each community
subgraphs = []
for community in communities:
    subgraphs.append(G.subgraph(community))
    
# print the number of nodes in each community
for i, subgraph in enumerate(subgraphs):
    print('Community '+str(i+1)+' has '+str(subgraph.number_of_nodes())+' nodes.')

##### <font color='darkblue'>Plot the Graph with Clusters Coloured</font>

In [ ]:
# create a dict with the gene_id as key and community membership list as value
communityDict = dict()

# loop through the communities
for i, community in enumerate(communities):
    # loop through the diseases in the community
    for gene_id in community:
        # add the disease and community to the dictionary
        communityDict[gene_id] = i

# plot the graph with the communities coloured
# create a list of 18 colors
communityColours = ['#1f77b4','#ff7f0e','#2ca02c']

# create a list of the node colours
nodeColours = [communityColours[communityDict[node]] for node in G.nodes()]

# create a list of the node labels
nodeLabels = {node:node for node in G.nodes()}

# plot the graph
import matplotlib.pyplot as plt

# set the figure size
plt.figure(figsize=(20,20))

# draw the graph separating nodes by their community

pos = nx.spring_layout(G, k=0.2, iterations=30, scale=1.5)
nx.draw(G, pos, node_color=nodeColours, with_labels=True, node_size=1000, font_size=12, width=0.5)

In [ ]:
# use enrichr to perform gene set enrichment analysis on the communities

#create a separate gene list from communityDict dictionary for each community
community1_genes = []
community2_genes = []
community3_genes = []

# loop through the dictionary
for gene_id, community in communityDict.items():
    # add the gene to the appropriate community list
    if community == 0:
        community1_genes.append(gene_id)
    elif community == 1:
        community2_genes.append(gene_id)
    elif community == 2:
        community3_genes.append(gene_id)

def communityORA(genes):
    # perform ORA against the Hallmark gene sets for each community
    enr = gp.enrichr(gene_list=genes, # perform enrichment analysis using gsea
                    gene_sets=['MSigDB_Hallmark_2020'],
                    organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                    outdir=None, # don't write to disk
                    )
    return enr

# perform ORA for each community
community1_enr = communityORA(community1_genes)
community2_enr = communityORA(community2_genes)
community3_enr = communityORA(community3_genes)

# print the top 10 results for each community using prettytable
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Community 1","Community 2","Community 3"]
for i in range(10):
    x.add_row([community1_enr.results['Term'][i],community2_enr.results['Term'][i],community3_enr.results['Term'][i]])
print(x)

##### <font color='darkblue'>Gene Set Enrichement Analysis Example</font>

In order to perform [GSEApy](https://gseapy.readthedocs.io/en/latest/index.html) we are going to use some suitable data. In this case some gene expression measurements for Parkinson's Disease from:

``Lewandowski NM et al.Polyamine pathway contributes to the pathogenesis of Parkinson disease.
Proc Natl Acad Sci U S A. 2010 Sep 28;107(39):16970-5. doi: 10.1073/pnas.1011751107.``

In [ ]:
# Note this is crude as we are simply using normalised gene expression values, not for example performing differential gene expression analysis
# we will return to this data in the coming weeks when we look at how we analyse gene expression data to find potentially important genes

# we first read in the gene expression data which was measured from people with Parkinson's disease and healthy controls
genExp = pd.read_csv('../data/expression/PD_Expr.tsv',sep='\t',header=0,index_col=False)

# # select rows where 'IDENTIFIER' is in the gene_ids list
# genExp = genExp.loc[genExp['IDENTIFIER'].isin(gene_symbols)]

# change column names 'ID_Ref' to 'Name' and 'IDENTIFIER' to 'GENES'
genExp = genExp.rename(columns={'ID_REF':'NAME','IDENTIFIER':'GENES'})

# drop the 'NAME' column
genExp = genExp.drop(labels='NAME',axis=1)

#make the GENES column the index
genExp = genExp.set_index('GENES')

# how big is the dataframe
print('The dataframe has '+str(genExp.shape[0])+' rows and '+str(genExp.shape[1])+' columns.\n')

# show the top of the dataframe
genExp.head()

In [ ]:
# we next need to specify which columns (samples are from PD patients and which are from controls)

control = ['GSM488132','GSM488118','GSM488116','GSM488114','GSM488112','GSM488131','GSM488117','GSM488115','GSM488113','GSM488111']
parkinsons = ['GSM488130','GSM488128','GSM488126','GSM488124','GSM488122','GSM488120','GSM488129','GSM488127','GSM488125','GSM488123','GSM488121','GSM488119']

# create a class list where the class is 'control' if the sample is in the control list and 'pd' if it is in the pd list
classes = []
for sample in genExp.columns:
    if sample in control:
        classes.append('control')
    elif sample in parkinsons:
        classes.append('parkinsons')
    else:
        pass

# print the number of samples in each class
print('There are '+str(classes.count('control'))+' control samples and '+str(classes.count('parkinsons'))+' PD samples.\n')

print(classes)

In [ ]:
# perform GSEA for each community against the union of the other commmunites

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# #list the available MSigDB libraries
# libraries = gp.get_library_name()

# #pick a random library from libraries using random.choice
# gene_sets = np.random.choice(libraries)

# #print the library name
# print('The library we will use is '+gene_sets)

gene_sets = 'KEGG_2019_Human'

#perform GSEA
gs_res = gp.gsea(data=genExp,
                 gene_sets=gene_sets,
                 cls= classes,
                 permutation_num=100,
                 outdir=None,
                 method='signal_to_noise',
                 threads=4, seed= 7)

gs_res.res2d.head()

In [ ]:
# plot the GSEA curves
terms = gs_res.res2d.Term
axs = gs_res.plot(terms[:5], show_ranking=False, legend_kws={'loc': (1.05, 0)}, )

In [ ]:
from gseapy import heatmap
# plotting heatmap
i = 2
genes = gs_res.res2d.Lead_genes[i].split(";")
# Make sure that ``ofname`` is not None, if you want to save your figure to disk
ax = heatmap(df = gs_res.heatmat.loc[genes], z_score=0, title=terms[i], figsize=(14,4))
# label the x axis with the class label of the sample
ax.set_xticklabels(classes,rotation=45,ha='right')
ax.plot()

In [ ]:
from gseapy import dotplot
# to save your figure, make sure that ``ofname`` is not None
ax = dotplot(gs_res.res2d,
             column="FDR q-val",
             title='KEGG_2021_Human',
             cmap=plt.cm.viridis,
             size=5,
             figsize=(4,5), cutoff=1)

##### <font color='darkblue'>(Optional) Using the PantherDB API for Enrichment Analysis</font>

PantherDB API details - [http://pantherdb.org/services/details.jsp](http://pantherdb.org/services/details.jsp)

Functionality and Parameter testing - [http://pantherdb.org/services/openAPISpec.jsp](http://pantherdb.org/services/openAPISpec.jsp)

This is quite hard work so you might decide it's simplest (and quicker) at this stage to use the website functionality above).

In [ ]:
# results are returned in JSON format so we need to load a Python module to handle this
import json

# the PantherDB API offers this function to find out what annotated resources it has available
query_url = 'http://pantherdb.org/services/oai/pantherdb/supportedannotdatasets'

# execute the query
result = ul.request.urlopen(query_url)

# load the results returning a Python dictionary
annotationSets = json.load(result)

annotations = annotationSets['search']['annotation_data_sets']['annotation_data_type']

# we can just iterate through these to see the annotation sources available
for i in annotations:
    print('Annotation Set Label = '+i['label']+', annotDataSet string to use below = '+i['id'])

In [ ]:
# using the list of Entrez Gene IDs generated above (entrezgene_id) create a query string for the API
# these need to be comma separated
genes = ','.join(map(str,gene_ids))

# use the PantherDB API - NB that GO:0008150 is the accession for the "Biological Process" clade of the Gene Ontology from above
query_url = "http://pantherdb.org/services/oai/pantherdb/enrich/overrep?&geneInputList="+genes+"&organism=9606&annotDataSet=GO:0008150&enrichmentTestType=FISHER&correction=FDR"

# capture the results (NB this returns in JSON format)
result = ul.request.urlopen(query_url)

# load the results from JSON to Python dictionary
enrichment_result = json.load(result)

# view the raw results
print(enrichment_result)

We're now going to format that into something human readable. There are many ways to do this, but this is a quick and (fairly) simple solution. Please do feel free to try your own.

In [ ]:
# extract the actual result component
results = enrichment_result['results']['result']

# how long is the background list (in this case it is the default, the whole genome)
print(len(results), "terms in reference list")

# we're going to print this in a nice looking ASCII table
from prettytable import PrettyTable

x = PrettyTable()

x.field_names = ["GO Term", "Expected", "Fold enrichment", "raw P value", "FDR", "Term label"]

# Sort in order of false discovery rate i.e. multiple testing correction
results.sort(key=lambda x: x['fdr'], reverse=False)

# show the top10 results
for r in results[:10]:
    fdr = r['fdr']
    if fdr < 0.05:
        # Print result line
        term_id = r['term'].get("id")
        if term_id is None:
            term_id = ""
        else:
            current_row = [
                term_id,
                str('{0:.3f}'.format(r['expected'])),  # Convert float to string for printing
                str('{0:.3f}'.format(r['fold_enrichment'])),
                str('{0:.3g}'.format(r['pValue'])),
                str('{0:.3g}'.format(r['fdr'])),
                r['term']["label"]
            ]
    x.add_row(current_row)

print(x)

# that was quite painful